## Setup

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
from supabase import create_client, Client
import pandas as pd
import dotenv
import os
import uuid
from datetime import datetime, timezone
import requests
from urllib.parse import urlparse, parse_qs



SERVICE_ACCOUNT_KEY = "./../serviceAccountKey.json"
FILE_PATH = "./../Documentation\SETS Team, RS, District Officers - SETS Dubar 2025 (Responses) (2).xlsx"

dotenv.load_dotenv()

True

In [2]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred)
db = firestore.client()
supabase = create_client(os.getenv('EXPO_PUBLIC_SUPABASE_URL'), os.getenv('EXPO_PUBLIC_SUPABASE_ANON_KEY'))

In [3]:
df = pd.read_excel(FILE_PATH)

## Preprocess

In [4]:
df.head()

,Timestamp,Email address,Full Name,Affiliation,Designation,Date of Birth,Sex,Photograph,Spouse's Name,Wedding Anniversary,...,Business Address,Business Website,About your business,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Phone,T Shirt Size,Shirt Size,Meal Preference,Blood Group
0,2024-12-19 10:03:04.673,gopi@fin1solutions.com,R GOPINATH,SETS Team,SETS SECRETARY,1972-02-25 00:00:00,Male,https://drive.google.com/open?id=18w_j4W8cQtuI...,VIJAYALAKSHMI,1998-11-02,...,NO 264 B BLOCK FLAT NO 4 ANBUR COLONY ANNA NAG...,www.fin1solutions.com,Fin 1 Solutions is a DSA for HDFC Bank for the...,TARUN GOPINATH,SON,7358543242,L,42,Non-Vegetarian,NaN
1,2024-12-26 19:22:13.109,rtn.uma1505@gmail.com,Rtn. Uma Yuvaraj,SETS Team,Ex AG,1980-05-15 00:00:00,Female,https://drive.google.com/open?id=1_zb_c7fblsNC...,Rtn. Yuvaraj,2020-03-24,...,.,notapplicable.com,..,Yuvaraj,Spouse,9361455621,M,0,Vegetarian,NaN
2,2024-12-26 19:35:50.786,advocatesuresh763@gmail.com,Sureshselvakumar,SETS Team,President,1965-05-31 00:00:00,Male,https://drive.google.com/open?id=1Yrt2omdhAWWu...,Subbulakshmi,1965-05-31,...,"FOM, 4 th floor, Jain citadel Apartment, Justi...",NaN,"My father is retired Judge, I handle all type ...",Subbulakshmi,Wife,9790708712,XXXL,48,Non-Vegetarian,NaN
3,2024-12-26 19:38:22.354,nokiasai@gmail.com,M Sesha Sai,SETS Team,DRCC,1965-08-22 00:00:00,Male,https://drive.google.com/open?id=1dK5I2dsGoxeW...,M Kamala,1989-05-19,...,62 Veerabhadran street. Nungambakkam. Chennai ...,www.farmguru.org,Farm Guru is a social initiative that educates...,M Kamala,Wife,9840546655,L,42,Non-Vegetarian,NaN
4,2024-12-27 11:10:49.283,bharatmpujara@gmail.com,Bharat Pujara,SETS Team,Chairman,1955-04-15 00:00:00,Male,https://drive.google.com/open?id=1wZj4WhcrnMn1...,Bharati pujara,1976-02-07,...,6/7 5th street T K salai Mylapore Chennai 600004,NaN,Suppling Raw material to paper mills for Recyc...,Bharati Pujara,Wife,9840938880,XXXL,50,Vegetarian,NaN


In [5]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_').str.replace("'", "")
df.rename(columns={
    "if_self-employed_:_business_name_/_if_salaried_:_company_name": "company_name",
    "designation": "sets_designation",
    "role_in_business_/_designation_in_company": "designation",
    "full_name": "name"}, inplace=True)
df.drop(columns=['timestamp'], inplace=True)

In [6]:
df["name_lower"] = df["name"].str.lower()
df["company_name_lower"] = df["company_name"].str.lower()
df["club_name_lower"] = df["club_name"].str.lower()
df["priority"] = 0
df['role'] = "member"
df["support"] = True
df.fillna("", inplace=True)

C:\Users\Anirudh\AppData\Local\Temp\ipykernel_10592\366255470.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


In [7]:
def convert_timestamp_to_string(timestamp):
    
    if pd.isna(timestamp):
        return ""
    
    if isinstance(timestamp, datetime):
        if timestamp.tzinfo is None:  # Check if the timestamp is naive
            timestamp = timestamp.replace(tzinfo=timezone.utc)
        else:
            timestamp = timestamp.astimezone(timezone.utc)
    formatted_string = timestamp.strftime('%Y-%m-%d %H:%M:%S%z')
    if formatted_string.endswith("+0000"):
        formatted_string = formatted_string[:-2] + ":00"
    return formatted_string

In [8]:
df.columns

Index(['email_address', 'name', 'affiliation', 'sets_designation',
       'date_of_birth', 'sex', 'photograph', 'spouses_name',
       'wedding_anniversary', 'phone', 'email', 'club_name', 'rotarian_since',
       'rotary_foundation_title', 'residential_address', 'company_name',
       'designation', 'type_of_business', 'business_address',
       'business_website', 'about_your_business', 'emergency_contact_name',
       'emergency_contact_relationship', 'emergency_contact_phone',
       't_shirt_size', 'shirt_size', 'meal_preference', 'blood_group',
       'name_lower', 'company_name_lower', 'club_name_lower', 'priority',
       'role', 'support'],
      dtype='object')

In [9]:
# datetime_columns = df.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns

In [10]:
datetime_columns = ['date_of_birth', 'wedding_anniversary']

In [11]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
df['wedding_anniversary'] = pd.to_datetime(df['date_of_birth'], errors='coerce')

In [12]:
for col in datetime_columns:
    df[col] = df[col].dt.tz_localize("UTC", ambiguous='NaT', nonexistent='NaT')

In [13]:
for col in datetime_columns:
    df[col] = df[col].apply(convert_timestamp_to_string)

In [14]:
data = df.to_dict(orient='records')

## Upload Image + Return Public URL

In [15]:
def download_file_from_google_drive(uid, photo_url):       
    parsed_url = urlparse(photo_url)
    query_params = parse_qs(parsed_url.query)
    file_id = query_params['id'][0]
    
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": file_id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    return save_response_content(uid, response)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None

def save_response_content(uid, response):   
    file_name = f"{uid}.png"  # Assuming the image format is JPEG
    file_content = response.content
    
    try:
        responseSupabase = supabase.storage.from_("members").upload(file_name, file_content)
        public_url = supabase.storage.from_("members").get_public_url(file_name)
        return public_url
    except Exception as e:
        print(e)


## Upsert Users

- Check for email existence in firebase auth, if it exists then update record in firestore
- If email does not exist then create new user in firebase auth and then create new record in firestore with doc id = user.uid

In [16]:
for user_data in data:
    email = user_data.get("email")
    if not email:
        print("Skipping record without email:", user_data)
        continue
    
    try:
        # Check if user exists in Firebase Auth
        user = auth.get_user_by_email(email)
        print(f"User exists: {email} (UID: {user.uid}). Updating record")
        
        # Update Firestore record
        user_data['id'] = user.uid
        try:
            public_url = download_file_from_google_drive(user_data['id'], user_data['photograph'])
            user_data['photograph'] = public_url
            # Upsert the record into Supabase
            response = supabase.table("members").upsert(user_data).execute()

            if len(response.data) <= 0:
                print(f"Error while upserting record for email {email}: {response}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")
        
    except auth.UserNotFoundError:
        print(f"User does not exist: {email}")
        
        # Create new user in Firebase Auth
        new_user = auth.create_user(email=email, password=email.split('@')[0] + '_password')
        print(f"Created new user: {email} (UID: {new_user.uid})")
        
        # Create new Firestore record
        user_data['id'] = user.uid
        try:
            public_url = download_file_from_google_drive(user_data['id'], user_data['photograph'])
            user_data['photograph'] = public_url
            
            response = supabase.table("members").upsert(user_data).execute()

            if len(response.data) <= 0:
                print(f"Error while upserting record for email {email}: {response}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")

User exists: gopi@fin1solutions.com (UID: MnEZWLNoTqXsSgvUSlH3Sr4I0A72). Updating record
Successfully upserted record for email gopi@fin1solutions.com
User exists: rtn.uma1505@gmail.com (UID: E3kekKeFkTNDdp6rgG6wtgFYmWY2). Updating record
Successfully upserted record for email rtn.uma1505@gmail.com
User exists: advocatesuresh763@yahoo.com (UID: 2DXGEuXDgdXBAAzRgnyLhCghD8D2). Updating record
Successfully upserted record for email advocatesuresh763@yahoo.com
User exists: rotaryclubsai@gmail.com (UID: Pntjifnvm5TmFVWpLvRRS5JRTEn1). Updating record
Successfully upserted record for email rotaryclubsai@gmail.com
User exists: bharatmpujara@gmail.com (UID: 6huDCteYDyfr2OXenMVVEQjMufs2). Updating record
Successfully upserted record for email bharatmpujara@gmail.com
User exists: sivailangovan@gmail.com (UID: ome8t3590fWIvbZgGZA3ZKWz7u43). Updating record
Successfully upserted record for email sivailangovan@gmail.com
User exists: palanivelghanesh@gmail.com (UID: 1Hhv2FiZpoQxojDWQWlCHzD6Cay1). Upd